In [1]:
import pandas as pd
total = pd.read_csv('./csv/total.csv')

/Users/YiYoung/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (8,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import pandas as pd
master = pd.read_csv('./csv/Master.csv')
product = pd.read_csv('./csv/Pruduct.csv')
session = pd.read_csv('./csv/Session.csv')
search1 = pd.read_csv('./csv/Search1.csv')
search2 = pd.read_csv('./csv/Search2.csv')
custom = pd.read_csv('./csv/Custom.csv')

## 1. master : master에서 product category id 별로 상품 개수 생성

In [3]:
category = master.groupby(['CLAC3_NM','CLAC2_NM','CLAC1_NM']).nunique()
category = category.drop(['CLAC3_NM','CLAC2_NM','CLAC1_NM'],axis=1)
category.reset_index(inplace=True) 
category.columns = ['CLAC3_NM','CLAC2_NM','CLAC1_NM','NUM_PD_C','NUM_PD_NM']
category.head()

,CLAC3_NM,CLAC2_NM,CLAC1_NM,NUM_PD_C,NUM_PD_NM
0,2단우산,우산/양산류,패션잡화,23,23
1,3단우산,우산/양산류,패션잡화,247,226
2,BB/파운데이션/컴팩트류,메이크업,화장품/뷰티케어,2184,2161
3,DIY가구,시공/DIY가구,가구,18,18
4,DIY완구,여아완구,완구,2,2


In [4]:
category.sort_values('NUM_PD_C',ascending=False).head()

,CLAC3_NM,CLAC2_NM,CLAC1_NM,NUM_PD_C,NUM_PD_NM
184,남성티셔츠,남성의류상의,남성의류,41650,39410
552,여성원피스,여성의류전신,여성의류,40048,37434
500,여성남방셔츠,여성의류상의,여성의류,34595,32750
573,여성티셔츠/탑,여성의류상의,여성의류,31087,29843
515,여성바지,여성의류하의,여성의류,22276,21492


In [186]:
category = category.drop(columns='NUM_PD_NM')

## 2. product : product인 상품구매 테이블과 merge

## product 전처리

In [5]:
new = []
for x in product.PD_BUY_CT:
    if type(x)!=int:
        x=x.strip("'")
        x=x.replace(',','')        
    new.append(x)
product['PD_BUY_CT'] = new
product['PD_BUY_CT']=product['PD_BUY_CT'].astype('int')
product.head()

new = []
for x in product.PD_BRA_NM:
    new.append(x.strip('[]'))
product['PD_BRA_NM'] = new


new2 = []
for x in product.PD_BUY_AM:
    new2.append(x.replace(",", ""))

product['PD_BUY_AM'] = new2
product['PD_BUY_AM']= product['PD_BUY_AM'].astype('int')

In [6]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5024906 entries, 0 to 5024905
Data columns (total 8 columns):
CLNT_ID      int64
SESS_ID      int64
HITS_SEQ     int64
PD_C         int64
PD_ADD_NM    object
PD_BRA_NM    object
PD_BUY_AM    int64
PD_BUY_CT    int64
dtypes: int64(6), object(2)
memory usage: 306.7+ MB


In [7]:
cproduct = master.merge(product, on='PD_C')

In [9]:
cproduct.head()

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_ID,SESS_ID,HITS_SEQ,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,1489284,2719478,269,색상:블랙|사이즈:SM[90] / 1개,언더아머(스포츠),49000,1
1,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,4671890,2659813,230,색상:블랙|사이즈:MD[95] / 1개,언더아머(스포츠),49000,1
2,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,559585,2330751,19,색상:블랙|사이즈:MD[95] / 1개,언더아머(스포츠),49000,1
3,62282,여자 플라워덧신 2족선물세트 17403-2set,속옷/양말/홈웨어,여성양말류,여성일반양말,2820547,733100,23,4개,안스엉클(양말),10000,4
4,61729,88A2933253배트맨스웨트티 - 블랙 / 130,유아동의류,유아의류상의,영유아티셔츠/탑,221797,68725,210,색상:블랙|사이즈:140 / 1개,래핑차일드,35000,1


In [8]:
cproduct_nunique = cproduct.groupby('CLAC3_NM').nunique()

In [11]:
cproduct_nunique

,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_ID,SESS_ID,HITS_SEQ,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
CLAC3_NM,,,,,,,,,,,,
2단우산,23,23,1,1,1,122,129,97,66,14,19,7
3단우산,247,226,1,1,1,1094,1156,256,290,31,104,8
BB/파운데이션/컴팩트류,2184,2161,1,1,1,44162,55075,485,4032,238,385,13
DIY가구,18,18,1,1,1,41,80,73,23,2,22,28
DIY완구,2,2,1,1,1,1,1,1,2,1,2,1
DSLR,54,50,1,1,1,292,492,172,9,5,93,2
LCD,3,3,1,1,1,22,22,23,4,3,6,1
LED,116,107,1,1,1,1004,1194,202,26,11,187,4
OLED,10,10,1,1,1,10,13,15,4,1,10,1


## CLNT_ID, SESS_ID, PD_ADD_NM, PD_BRA_NM

In [9]:
category_product = cproduct_nunique[['CLNT_ID','SESS_ID','PD_ADD_NM','PD_BRA_NM']]
category_product.reset_index(inplace=True)
category_product.columns = ['CLAC3_NM','NUM_CLNT_ID','NUM_SESS_ID','NUM_ADD_NM','NUM_BRA_NM']

In [10]:
category_product.head()

,CLAC3_NM,NUM_CLNT_ID,NUM_SESS_ID,NUM_ADD_NM,NUM_BRA_NM
0,2단우산,122,129,66,14
1,3단우산,1094,1156,290,31
2,BB/파운데이션/컴팩트류,44162,55075,4032,238
3,DIY가구,41,80,23,2
4,DIY완구,1,1,2,1


## HITS_SEQ, PD_BUY_AM, PD_BUY_CT


In [11]:
cproduct_avg = cproduct.groupby('CLAC3_NM').mean()
cproduct_avg = cproduct_avg[['HITS_SEQ','PD_BUY_AM','PD_BUY_CT']]
cproduct_avg.reset_index(inplace=True)
cproduct_avg= round(cproduct_avg,2)
cproduct_avg.columns=['CLAC3_NM','AVG_HITS','AVG_PRICE','AVG_CNT']
cproduct_avg.head()

,CLAC3_NM,AVG_HITS,AVG_PRICE,AVG_CNT
0,2단우산,79.15,12595.06,1.27
1,3단우산,75.48,14239.82,1.12
2,BB/파운데이션/컴팩트류,63.60,51865.73,1.09
3,DIY가구,59.68,2488420.22,13.63
4,DIY완구,23.00,16500.00,1.00


## total --> HITS_SEQ, PD_BUY_AM, PD_BUY_CT

In [12]:
cproduct_total = cproduct.groupby('CLAC3_NM').sum()
cproduct_total = cproduct_total[['HITS_SEQ','PD_BUY_AM','PD_BUY_CT']]
cproduct_total.reset_index(inplace=True)
cproduct_total.columns=['CLAC3_NM','SUM_HITS','SUM_PRICE','SUM_CNT']
cproduct_total.head()

,CLAC3_NM,SUM_HITS,SUM_PRICE,SUM_CNT
0,2단우산,12822,2040400,206
1,3단우산,110574,20861330,1637
2,BB/파운데이션/컴팩트류,4046376,3299956970,69417
3,DIY가구,6744,281191485,1540
4,DIY완구,46,33000,2


## 3. custom : cproduct + custom
* cproduct = master + product
* cuscproduct = cproduct + custom

In [13]:
cuscproduct = cproduct.merge(custom, on ='CLNT_ID',how='outer')

In [208]:
cuscproduct.count()

PD_C           5024906
PD_NM          5024906
CLAC1_NM       5024906
CLAC2_NM       5024906
CLAC3_NM       5024906
CLNT_ID        5024906
SESS_ID        5024906
HITS_SEQ       5024906
PD_ADD_NM      5024906
PD_BRA_NM      5024906
PD_BUY_AM      5024906
PD_BUY_CT      5024906
CLNT_GENDER    3988587
CLNT_AGE       3988587
dtype: int64

## 여성과 남성

In [14]:
F = cuscproduct.query('CLNT_GENDER == "F"').groupby('CLAC3_NM').count()['CLNT_GENDER']

In [15]:
M = cuscproduct.query('CLNT_GENDER == "M"').groupby('CLAC3_NM').count()['CLNT_GENDER']

In [16]:
gender = pd.concat([F,M],axis=1)
gender.reset_index(inplace=True)
gender.columns = ['CLAC3_NM','F','M']

/Users/YiYoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [79]:
gender.head()

,CLAC3_NM,F,M
0,2단우산,108.0,14.0
1,3단우산,1000.0,123.0
2,BB/파운데이션/컴팩트류,49829.0,2511.0
3,DIY가구,41.0,41.0
4,DSLR,100.0,52.0


## 연령대 10,20,30,40,50,60,70,80

In [18]:
cuscproduct['CLNT_AGE'].unique()

array([40., 30., nan, 50., 20., 60., 10., 70., 80.])

In [19]:
AGE10 = cuscproduct.query('CLNT_AGE == 10').groupby('CLAC3_NM').count()['CLNT_AGE']
AGE20 = cuscproduct.query('CLNT_AGE == 20').groupby('CLAC3_NM').count()['CLNT_AGE']
AGE30 = cuscproduct.query('CLNT_AGE == 30').groupby('CLAC3_NM').count()['CLNT_AGE']
AGE40 = cuscproduct.query('CLNT_AGE == 40').groupby('CLAC3_NM').count()['CLNT_AGE']
AGE50 = cuscproduct.query('CLNT_AGE == 50').groupby('CLAC3_NM').count()['CLNT_AGE']
AGE60 = cuscproduct.query('CLNT_AGE == 60').groupby('CLAC3_NM').count()['CLNT_AGE']
AGE70 = cuscproduct.query('CLNT_AGE == 70').groupby('CLAC3_NM').count()['CLNT_AGE']
AGE80 = cuscproduct.query('CLNT_AGE == 80').groupby('CLAC3_NM').count()['CLNT_AGE']


In [20]:
AGE = pd.concat([AGE10,AGE20,AGE30,AGE40,AGE50,AGE60,AGE70,AGE80],axis=1)
AGE.reset_index(inplace=True)
AGE.columns = ['CLAC3_NM','AGE10','AGE20','AGE30','AGE40','AGE50','AGE60','AGE70','AGE80']

/Users/YiYoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [21]:
AGE.head()

,CLAC3_NM,AGE10,AGE20,AGE30,AGE40,AGE50,AGE60,AGE70,AGE80
0,2단우산,NaN,2.0,21.0,90.0,9.0,NaN,NaN,NaN
1,3단우산,1.0,57.0,351.0,617.0,88.0,8.0,1.0,NaN
2,BB/파운데이션/컴팩트류,341.0,11514.0,23000.0,14754.0,2365.0,326.0,34.0,6.0
3,DIY가구,NaN,NaN,27.0,41.0,14.0,NaN,NaN,NaN
4,DSLR,2.0,22.0,72.0,48.0,8.0,NaN,NaN,NaN


In [114]:
c1 = pd.DataFrame(cuscproduct['CLAC3_NM'].unique())

In [110]:
c2 = pd.DataFrame(AGE['CLAC3_NM'])

In [115]:
c1.equals(c2)

False

## Session 
* sess_seq 평균
* sess_dt --> 월별 빈도수 & 요일별 빈도수
* tot_pag_view_ct --> 평균 // 월별 페이지 빈도수 & 요일별 페이지 빈도수
* tot_sess_hr_v --> 평균 // 
* DVC_CTG_NM --> 기기별 빈도수
* ZON, CITY 별 빈도수 

In [91]:
session = pd.read_csv('./csv/Session.csv')
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,"1,890",mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,"1,604",mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,"1,035",mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298,mobile,Seoul,Seoul


## session 전처리

In [22]:
new = []
for x in session.TOT_SESS_HR_V:
    if type(x)!=int and type(x)!=float:
        x=x.replace(',','')   
    new.append(x)
session['TOT_SESS_HR_V'] = new
session['TOT_SESS_HR_V'] = [int(i) if pd.notnull(i) else i for i in session['TOT_SESS_HR_V']]

session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,1890.0,mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,1604.0,mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632.0,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,1035.0,mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298.0,mobile,Seoul,Seoul


In [23]:
session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2712907 entries, 0 to 2712906
Data columns (total 9 columns):
CLNT_ID            int64
SESS_ID            int64
SESS_SEQ           int64
SESS_DT            int64
TOT_PAG_VIEW_CT    float64
TOT_SESS_HR_V      float64
DVC_CTG_NM         object
ZON_NM             object
CITY_NM            object
dtypes: float64(2), int64(4), object(3)
memory usage: 186.3+ MB


In [27]:
import datetime
session['SESS_DT']=session.SESS_DT.apply(lambda x: datetime.datetime.strptime(str(x),'%Y%m%d'))
opt = session.SESS_DT.dt
opt_prop = pd.DataFrame({'year':opt.year,
             'month':opt.month,
             'day':opt.day,
              'weekofyear':opt.weekofyear,
             'dayofyear':opt.dayofyear,
              'weekday':opt.weekday,
              'quarter':opt.quarter
             })
opt_prop.head()

,year,month,day,weekofyear,dayofyear,weekday,quarter
0,2018,5,9,19,129,2,2
1,2018,6,11,24,162,0,2
2,2018,6,24,25,175,6,2
3,2018,9,13,37,256,3,3
4,2018,4,12,15,102,3,2


In [28]:
session = pd.concat([session,opt_prop],axis=1)
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,year,month,day,weekofyear,dayofyear,weekday,quarter
0,5873599,8641867,9,2018-05-09,82.0,1890.0,mobile,Daejeon,Daejeon,2018,5,9,19,129,2,2
1,5873599,6616320,21,2018-06-11,105.0,1604.0,mobile,Busan,Busan,2018,6,11,24,162,0,2
2,5873599,5886172,40,2018-06-24,41.0,632.0,mobile,Daejeon,Daejeon,2018,6,24,25,175,6,2
3,5873884,1050889,15,2018-09-13,160.0,1035.0,mobile,Gyeonggi-do,Anyang,2018,9,13,37,256,3,3
4,5874461,10298270,5,2018-04-12,13.0,298.0,mobile,Seoul,Seoul,2018,4,12,15,102,3,2


In [29]:
subtotal = cuscproduct.merge(session, on=['CLNT_ID','SESS_ID'],how = 'outer')

In [30]:
subtotal.head()


,PD_C,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,CLNT_ID,SESS_ID,HITS_SEQ,PD_ADD_NM,PD_BRA_NM,...,DVC_CTG_NM,ZON_NM,CITY_NM,year,month,day,weekofyear,dayofyear,weekday,quarter
0,64382,언더아머 남성 UA HG 아머 모크 LS 1289559-001 - 블랙 / MD[95],스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,1489284,2719478,269,색상:블랙|사이즈:SM[90] / 1개,언더아머(스포츠),...,mobile,Seoul,Seoul,2018,8,15,33,227,2,3
1,301738,아디다스 D2M K 숏 (CV3341) 여성 숏반바지/숏팬츠 - 100,스포츠패션,여성일반스포츠의류,여성일반스포츠바지,1489284,2719478,309,사이즈:085 / 1개,아디다스(의류),...,mobile,Seoul,Seoul,2018,8,15,33,227,2,3
2,676611,DTP 스판 스탠드 에어컨커버 (15종택1) - 레이첼(체조요정),침구/수예,수예소품,거실수예소품,1489284,6419731,131,선택:스텔라(2013~2016삼성.LG신형) / 1개,쉬즈홈,...,mobile,Seoul,Seoul,2018,6,15,24,166,4,2
3,841908,민트 민 티스푼 1P,식기/조리기구,그릇/식기,디저트포크/스푼,1489284,6824467,38,5개,골든벨금속,...,mobile,Seoul,Seoul,2018,6,8,23,159,4,2
4,841902,프리마 민 롱스푼 1P,식기/조리기구,그릇/식기,디저트포크/스푼,1489284,6824467,38,3개,골든벨금속,...,mobile,Seoul,Seoul,2018,6,8,23,159,4,2


## mean

In [31]:
subtotal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5024906 entries, 0 to 5024905
Data columns (total 28 columns):
PD_C               int64
PD_NM              object
CLAC1_NM           object
CLAC2_NM           object
CLAC3_NM           object
CLNT_ID            int64
SESS_ID            int64
HITS_SEQ           int64
PD_ADD_NM          object
PD_BRA_NM          object
PD_BUY_AM          int64
PD_BUY_CT          int64
CLNT_GENDER        object
CLNT_AGE           float64
SESS_SEQ           int64
SESS_DT            datetime64[ns]
TOT_PAG_VIEW_CT    float64
TOT_SESS_HR_V      float64
DVC_CTG_NM         object
ZON_NM             object
CITY_NM            object
year               int64
month              int64
day                int64
weekofyear         int64
dayofyear          int64
weekday            int64
quarter            int64
dtypes: datetime64[ns](1), float64(3), int64(14), object(10)
memory usage: 1.1+ GB


In [99]:
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,year,month,day,weekofyear,dayofyear,weekday,quarter
0,5873599,8641867,9,2018-05-09,82.0,1890.0,mobile,Daejeon,Daejeon,2018,5,9,19,129,2,2
1,5873599,6616320,21,2018-06-11,105.0,1604.0,mobile,Busan,Busan,2018,6,11,24,162,0,2
2,5873599,5886172,40,2018-06-24,41.0,632.0,mobile,Daejeon,Daejeon,2018,6,24,25,175,6,2
3,5873884,1050889,15,2018-09-13,160.0,1035.0,mobile,Gyeonggi-do,Anyang,2018,9,13,37,256,3,3
4,5874461,10298270,5,2018-04-12,13.0,298.0,mobile,Seoul,Seoul,2018,4,12,15,102,3,2


### SESSION AVG
* sess_seq 평균
* tot_pag_view_ct --> 평균 // 월별 페이지 빈도수 & 요일별 페이지 빈도수
* tot_sess_hr_v --> 평균 //


* sess_dt --> 월별 빈도수 & 요일별 빈도수
* DVC_CTG_NM --> 기기별 빈도수
* ZON, CITY 별 빈도수 

In [32]:
subtotal_avg = subtotal.groupby('CLAC3_NM').mean()
subtotal_avg = subtotal_avg[['SESS_SEQ','TOT_PAG_VIEW_CT','TOT_SESS_HR_V']]
subtotal_avg.reset_index(inplace=True)
subtotal_avg= round(subtotal_avg,2)
subtotal_avg.columns=['CLAC3_NM','AVG_SESS_SEQ','AVG_TOT_PAG_VIEW_CT','AVG_TOT_SESS_HR_V']
subtotal_avg.head()

,CLAC3_NM,AVG_SESS_SEQ,AVG_TOT_PAG_VIEW_CT,AVG_TOT_SESS_HR_V
0,2단우산,210.80,103.27,1976.06
1,3단우산,209.34,92.18,1744.28
2,BB/파운데이션/컴팩트류,143.84,78.78,1435.65
3,DIY가구,70.52,76.54,2144.88
4,DIY완구,1.00,22.00,783.00


## 기기별 빈도수 DVC_CTG_NM

In [33]:
session['DVC_CTG_NM'].unique()

array(['mobile', 'desktop', 'tablet'], dtype=object)

In [71]:
mobile = subtotal.query("DVC_CTG_NM == 'mobile'").groupby('CLAC3_NM').count()['DVC_CTG_NM']
desktop = subtotal.query("DVC_CTG_NM == 'desktop'").groupby('CLAC3_NM').count()['DVC_CTG_NM']
tablet = subtotal.query("DVC_CTG_NM == 'tablet'").groupby('CLAC3_NM').count()['DVC_CTG_NM']

In [72]:
DVC= pd.concat([mobile,desktop,tablet],axis=1)
DVC.reset_index(inplace=True)
DVC.columns = ['CLAC3_NM','mobile','desktop','tablet']

/Users/YiYoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [73]:
DVC.head(10)

,CLAC3_NM,mobile,desktop,tablet
0,2단우산,124.0,36.0,2.0
1,3단우산,1148.0,309.0,8.0
2,BB/파운데이션/컴팩트류,53833.0,9567.0,225.0
3,DIY가구,88.0,25.0,NaN
4,DIY완구,NaN,2.0,NaN
5,DSLR,378.0,244.0,5.0
6,LCD,20.0,3.0,NaN
7,LED,1023.0,341.0,5.0
8,OLED,11.0,5.0,NaN
9,PC부품,2794.0,853.0,26.0


In [36]:
DVC.head()

,CLAC3_NM,mobile,desktop,tablet
0,2단우산,124.0,36.0,2.0
1,3단우산,1148.0,309.0,8.0
2,BB/파운데이션/컴팩트류,53833.0,9567.0,225.0
3,DIY가구,88.0,25.0,NaN
4,DIY완구,NaN,2.0,NaN


### sess_dt --> 월별 빈도수 빈도수

In [121]:
session['month'].unique()

array([5, 6, 9, 4, 8, 7])

In [149]:
April = subtotal.query("month == 4").groupby('CLAC3_NM').count()['month']
May = subtotal.query("month == 5").groupby('CLAC3_NM').count()['month']
June = subtotal.query("month == 6").groupby('CLAC3_NM').count()['month']
July = subtotal.query("month == 7").groupby('CLAC3_NM').count()['month']
August = subtotal.query("month == 8").groupby('CLAC3_NM').count()['month']
September = subtotal.query("month == 9").groupby('CLAC3_NM').count()['month']

Monthly = pd.concat([April,May,June,July,August,September],axis=1)
Monthly.reset_index(inplace=True)
Monthly.columns = ['CLAC3_NM','April','May','June','July','August','September']

/Users/YiYoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [150]:
Monthly.head()

,CLAC3_NM,April,May,June,July,August,September
0,2단우산,47.0,31.0,43.0,24.0,14.0,3.0
1,3단우산,342.0,330.0,257.0,233.0,189.0,114.0
2,BB/파운데이션/컴팩트류,10329.0,12268.0,11003.0,10246.0,10519.0,9260.0
3,DIY가구,15.0,20.0,29.0,21.0,18.0,10.0
4,DIY완구,NaN,NaN,NaN,NaN,NaN,2.0


## ZON 별 빈도수 

In [124]:
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,year,month,day,weekofyear,dayofyear,weekday,quarter
0,5873599,8641867,9,2018-05-09,82.0,1890.0,mobile,Daejeon,Daejeon,2018,5,9,19,129,2,2
1,5873599,6616320,21,2018-06-11,105.0,1604.0,mobile,Busan,Busan,2018,6,11,24,162,0,2
2,5873599,5886172,40,2018-06-24,41.0,632.0,mobile,Daejeon,Daejeon,2018,6,24,25,175,6,2
3,5873884,1050889,15,2018-09-13,160.0,1035.0,mobile,Gyeonggi-do,Anyang,2018,9,13,37,256,3,3
4,5874461,10298270,5,2018-04-12,13.0,298.0,mobile,Seoul,Seoul,2018,4,12,15,102,3,2


In [125]:
zon = list(session['ZON_NM'].unique())

zon_list = []
for i in zon:
    zon_list.append(subtotal.query("ZON_NM == '%s'"%i).groupby('CLAC3_NM').count()['ZON_NM'])
   

In [170]:
NUM_ZON=pd.concat([i for i in zon_list],axis=1)

NUM_ZON.columns = ['ZON_%s'%i for i in zon]
NUM_ZON.reset_index(inplace=True)
NUM_ZON=NUM_ZON.rename(columns = {'index':'CLAC3_NM'})


/Users/YiYoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [171]:
NUM_ZON.head()

,CLAC3_NM,ZON_Daejeon,ZON_Busan,ZON_Gyeonggi-do,ZON_Seoul,ZON_Gyeongsangbuk-do,ZON_Jeollabuk-do,ZON_Chungcheongbuk-do,ZON_Gangwon-do,ZON_Chungcheongnam-do,ZON_Incheon,ZON_Daegu,ZON_Gwangju,ZON_Gyeongsangnam-do,ZON_Jeju-do,ZON_Jeollanam-do,ZON_Ulsan
0,2단우산,4.0,14.0,37.0,58.0,4.0,1.0,5.0,2.0,1.0,7.0,7.0,4.0,11.0,NaN,1.0,6.0
1,3단우산,39.0,159.0,269.0,565.0,36.0,33.0,31.0,27.0,33.0,35.0,69.0,30.0,70.0,6.0,24.0,39.0
2,BB/파운데이션/컴팩트류,1346.0,6491.0,10331.0,26906.0,1926.0,1605.0,1078.0,1057.0,1414.0,2341.0,2384.0,1385.0,2695.0,687.0,1178.0,801.0
3,DIY가구,4.0,10.0,30.0,59.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN
4,DIY완구,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## city별 빈도수

In [128]:
session['CITY_NM'].unique()

array(['Daejeon', 'Busan', 'Anyang', 'Seoul', 'Gumi-si', 'Buan-gun',
       'Cheongju-si', 'Chuncheon-si', 'Cheonan-si', 'Incheon', 'Daegu',
       'Wonju-si', 'Seongnam-si', 'Goyang-si', 'Gwangmyeong-si',
       'Uijeongbu-si', 'Namyangju-si', 'Jeonju-si', 'Cheorwon-gun',
       'Hwaseong-si', 'Gwangju', 'Gimhae-si', 'Tongyeong-si',
       'Seogwipo-si', 'Ansan-si', 'Naju-si', 'Ulsan', 'Anseong',
       'Gimcheon-si', 'Yongin-si', 'Namwon-si', 'Bucheon-si', 'Gunsan-si',
       'Yeosu-si', 'Sacheon-si', 'Osan-si', 'Dangjin-si', 'Gunpo-si',
       'Hanam-si', 'Gongju-si', 'Seocheon-gun', 'Yeongi-gun',
       'Siheung-si', 'Paju-si', 'Yangsan-si', 'Gyeongju-si',
       'Goseong-gun', 'Iksan', 'Yeongcheon-si', 'Suncheon-si',
       'Pohang-si', 'Pyeongchang-gun', 'Suwon-si', 'Jincheon-gun',
       'Nonsan-si', 'Mokpo-si', 'Muan-gun', 'Haman-gun', 'Geoje-si',
       'Chilgok-gun', 'Hampyeong-gun', 'Gangneung-si', 'Ulleung-gun',
       'Icheon-si', 'Chungju-si', 'Pyeongtaek-si', 'Taean-gun'

In [168]:
city = list(session['CITY_NM'].unique())

city_list = []
for i in city:
    city_list.append(subtotal.query("CITY_NM == '%s'"%i).groupby('CLAC3_NM').count()['CITY_NM'])

In [169]:
NUM_CITY=pd.concat([i for i in city_list],axis=1)
NUM_CITY.columns = ['CITY_%s'%i for i in city]
NUM_CITY.reset_index(inplace=True)
NUM_CITY=NUM_CITY.rename(columns = {'index':'CLAC3_NM'})

/Users/YiYoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [167]:
NUM_CITY.head()

,index,CITY_Daejeon,CITY_Busan,CITY_Anyang,CITY_Seoul,CITY_Gumi-si,CITY_Buan-gun,CITY_Cheongju-si,CITY_Chuncheon-si,CITY_Cheonan-si,...,CITY_Sancheong-gun,CITY_Hoengseong-gun,CITY_Cheongdo-gun,CITY_Goesan-gun,CITY_Jindo-gun,CITY_Gurye-gun,CITY_Cheongyang-gun,CITY_(not set),CITY_Gunwi-gun,CITY_Masan
0,2단우산,4.0,14.0,3.0,58.0,1.0,NaN,2.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3단우산,39.0,159.0,22.0,565.0,13.0,NaN,17.0,4.0,9.0,...,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,BB/파운데이션/컴팩트류,1346.0,6491.0,718.0,26906.0,344.0,19.0,527.0,262.0,332.0,...,6.0,21.0,21.0,13.0,18.0,24.0,18.0,1.0,14.0,NaN
3,DIY가구,4.0,10.0,3.0,59.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DIY완구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 통합

In [187]:
del final

In [188]:
final=category.merge(category_product,on='CLAC3_NM',how='outer')
final=final.merge(cproduct_avg,on='CLAC3_NM',how='outer')

In [189]:
final=final.merge(cproduct_total,on='CLAC3_NM',how='outer')
final=final.merge(gender,on='CLAC3_NM',how='outer')

In [190]:
final.columns

Index(['CLAC3_NM', 'CLAC2_NM', 'CLAC1_NM', 'NUM_PD_C', 'NUM_CLNT_ID',
       'NUM_SESS_ID', 'NUM_ADD_NM', 'NUM_BRA_NM', 'AVG_HITS', 'AVG_PRICE',
       'AVG_CNT', 'SUM_HITS', 'SUM_PRICE', 'SUM_CNT', 'F', 'M'],
      dtype='object')

In [191]:
final=final.merge(AGE,on='CLAC3_NM',how='outer')
final=final.merge(subtotal_avg,on='CLAC3_NM',how='outer')
final=final.merge(DVC,on='CLAC3_NM',how='outer')

In [192]:
final=final.merge(Monthly,on='CLAC3_NM',how='outer')
final=final.merge(NUM_ZON,on='CLAC3_NM',how='outer')
final=final.merge(NUM_CITY,on='CLAC3_NM',how='outer')

In [193]:
final.head(10)

,CLAC3_NM,CLAC2_NM,CLAC1_NM,NUM_PD_C,NUM_CLNT_ID,NUM_SESS_ID,NUM_ADD_NM,NUM_BRA_NM,AVG_HITS,AVG_PRICE,...,CITY_Sancheong-gun,CITY_Hoengseong-gun,CITY_Cheongdo-gun,CITY_Goesan-gun,CITY_Jindo-gun,CITY_Gurye-gun,CITY_Cheongyang-gun,CITY_(not set),CITY_Gunwi-gun,CITY_Masan
0,2단우산,우산/양산류,패션잡화,23,122,129,66,14,79.15,12595.06,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3단우산,우산/양산류,패션잡화,247,1094,1156,290,31,75.48,14239.82,...,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,BB/파운데이션/컴팩트류,메이크업,화장품/뷰티케어,2184,44162,55075,4032,238,63.60,51865.73,...,6.0,21.0,21.0,13.0,18.0,24.0,18.0,1.0,14.0,NaN
3,DIY가구,시공/DIY가구,가구,18,41,80,23,2,59.68,2488420.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DIY완구,여아완구,완구,2,1,1,2,1,23.00,16500.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DSLR,카메라/캠코더,영상/음향가전,54,292,492,9,5,59.88,784287.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LCD,TV,영상/음향가전,3,22,22,4,3,67.00,605645.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LED,TV,영상/음향가전,116,1004,1194,26,11,57.04,562687.75,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
8,OLED,TV,영상/음향가전,10,10,13,4,1,91.44,3442923.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PC부품,컴퓨터주변기기,컴퓨터,827,2495,2824,313,86,59.19,28746.18,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.0,1.0,NaN


In [194]:
final_columns= [print(i) for i in final.columns]

CLAC3_NM
CLAC2_NM
CLAC1_NM
NUM_PD_C
NUM_CLNT_ID
NUM_SESS_ID
NUM_ADD_NM
NUM_BRA_NM
AVG_HITS
AVG_PRICE
AVG_CNT
SUM_HITS
SUM_PRICE
SUM_CNT
F
M
AGE10
AGE20
AGE30
AGE40
AGE50
AGE60
AGE70
AGE80
AVG_SESS_SEQ
AVG_TOT_PAG_VIEW_CT
AVG_TOT_SESS_HR_V
mobile
desktop
tablet
April
May
June
July
August
September
ZON_Daejeon
ZON_Busan
ZON_Gyeonggi-do
ZON_Seoul
ZON_Gyeongsangbuk-do
ZON_Jeollabuk-do
ZON_Chungcheongbuk-do
ZON_Gangwon-do
ZON_Chungcheongnam-do
ZON_Incheon
ZON_Daegu
ZON_Gwangju
ZON_Gyeongsangnam-do
ZON_Jeju-do
ZON_Jeollanam-do
ZON_Ulsan
CITY_Daejeon
CITY_Busan
CITY_Anyang
CITY_Seoul
CITY_Gumi-si
CITY_Buan-gun
CITY_Cheongju-si
CITY_Chuncheon-si
CITY_Cheonan-si
CITY_Incheon
CITY_Daegu
CITY_Wonju-si
CITY_Seongnam-si
CITY_Goyang-si
CITY_Gwangmyeong-si
CITY_Uijeongbu-si
CITY_Namyangju-si
CITY_Jeonju-si
CITY_Cheorwon-gun
CITY_Hwaseong-si
CITY_Gwangju
CITY_Gimhae-si
CITY_Tongyeong-si
CITY_Seogwipo-si
CITY_Ansan-si
CITY_Naju-si
CITY_Ulsan
CITY_Anseong
CITY_Gimcheon-si
CITY_Yongin-si
CITY_Namwon-si
C

In [195]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898 entries, 0 to 897
Columns: 215 entries, CLAC3_NM to CITY_Masan
dtypes: float64(204), int64(8), object(3)
memory usage: 1.5+ MB


In [196]:
final.nunique()

CLAC3_NM                898
CLAC2_NM                128
CLAC1_NM                 37
NUM_PD_C                576
NUM_CLNT_ID             767
NUM_SESS_ID             782
NUM_ADD_NM              557
NUM_BRA_NM              208
AVG_HITS                826
AVG_PRICE               898
AVG_CNT                 129
SUM_HITS                898
SUM_PRICE               898
SUM_CNT                 809
F                       755
M                       495
AGE10                    92
AGE20                   384
AGE30                   665
AGE40                   671
AGE50                   385
AGE60                   158
AGE70                    57
AGE80                    10
AVG_SESS_SEQ            883
AVG_TOT_PAG_VIEW_CT     843
AVG_TOT_SESS_HR_V       891
mobile                  775
desktop                 587
tablet                  126
                       ... 
CITY_Gokseong-gun        20
CITY_Yeongdong-gun       27
CITY_Jeungpyeong-gun     28
CITY_Hwasun-gun          42
CITY_Sinan-gun      

In [197]:
final.to_csv('podol_final.csv')